# Chem 277B FA23 Tutorial 9
____
## Outline 

* Savio Account Setup
* MNIST Dataset for HW7
* Big picture of the ANI project
* Understanding the data structure of ANI dataset
* Implementing atomic environment vector (AEV) calculator in ANI models with numpy
* ModuleDict

### References
* ANI-1: an extensible neural network potential with DFT accuracy at force field computational cost<br> ([Article](https://pubs.rsc.org/en/content/articlepdf/2017/sc/c6sc05720a), [SI](http://www.rsc.org/suppdata/c6/sc/c6sc05720a/c6sc05720a1.pdf), [Dataset](https://doi.org/10.6084/m9.figshare.c.3846712.v1)),

____


## 0. Collecting Questions for Next Week

* 
* 

____

## 1. Savio Account Setup

* request a savio account: https://docs-research-it.berkeley.edu/services/high-performance-computing/getting-account/#setting-up-a-mybrc-user-portal-account
* request to join the project named `ic_chem277b` (maybe not available at this moment. will send further notice.)

* resources about logging in and OTP setup: https://docs-research-it.berkeley.edu/services/high-performance-computing/user-guide/logging-brc-clusters/

We will cover more about requesting gpu nodes and submitting jobs next time. Stay Tuned!
___

## 2. MNIST Dataset

In this hw, you are going to explore the famouse MNIST Dataset. Here, we will peak into this dataset to see what's in it and how we can make predictions based on the input.

In [ ]:
import pickle
(train_X, train_y), (test_X, test_y) = pickle.load(open("mnist.pkl", "rb"))
 
#shape of dataset
print('X_train: ' + str(train_X.shape))
print('Y_train: ' + str(train_y.shape))
print('X_test:  '  + str(test_X.shape))
print('Y_test:  '  + str(test_y.shape))
 
#plotting
import matplotlib.pyplot as plt
%matplotlib inline
plt.figure()
for i in range(9):  
    plt.subplot(3,3,i+1)
    plt.imshow(train_X[i], cmap=plt.get_cmap('gray'))

_______
## 3. Understanding data organization

https://github.com/isayev/ANI1_dataset.git
Adapted from example_data_sampler.py

*Run in terminal*

1. First, let's navigate to the scratch folder where we don't have a data cap [NEED TO CHANGE USER NAME]  

`cd /global/scratch/users/user_name/`  
  
2. Then, let's create a new folder for our project  

`mkdir ANI-dataset`

3. Then, we can go in this folder and download the dataset from the github page  

`cd ani-project`  
`wget https://figshare.com/ndownloader/files/9057631`  

Once the download is finished, we can rename the file from `9057631` to be `ANI-1_release.tar.gz`.  

4. The tar.gz file is like a zip file where all the data are compressed, we can extract this dataset through

`tar -xvzf ANI-1_release.tar.gz`

Now, in your scratch folder, you should be able to click into a folder called "ANI-1_release" and you will find your data in it.

5. Upload the `pyanitools.py` file to the same folder `ANI-dataset`

6. Here, we basically have set up the dataset and necessary tools to process the dataset. We need one final step of creating a new virtual environment for this ANI project. [NEED TO CHANGE USER NAME] 

`conda create -p /global/scratch/users/user_name/envs/ani python==3.10`  
`conda activate /global/scratch/users/user_name/envs/ani`

7. Let's install all the necessary packages for the project

`conda install numpy matplotlib seaborn scipy scikit-learn ipykernel`  
`ipython kernel install --user --name=ani`  
`conda install -c conda-forge torchani`  
`pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118`


In [1]:
import sys
sys.path.append("/global/scratch/users/kysun/ANI-dataset")
import pyanitools as pya

# Set the HDF5 file containing the data
hdf5file = '/global/scratch/users/kysun/ANI-dataset/ANI-1_release/ani_gdb_s01.h5'

# Construct the data loader class
adl = pya.anidataloader(hdf5file)

# Print the species of the data set one by one
for data in adl:
#     print(data.keys())

    # Extract the data
    P = data['path']
    X = data['coordinates']
    E = data['energies']
    S = data['species']
    sm = data['smiles']

    # Print the data
    print("Path:   ", P)
    print("  Smiles:      ","".join(sm))
    print("  Symbols:     ", S)
    print("  Coordinates: ", X.shape)
    print("  Energies:    ", E.shape, "\n")

# Closes the H5 data file
adl.cleanup()

ModuleNotFoundError: No module named 'pyanitools'

### Use Iterator to get data

In [ ]:
from pyanitools import anidataloader
data = anidataloader("/global/scratch/users/kysun/ANI-dataset/ANI-1_release/ani_gdb_s01.h5")
data_iter = data.__iter__()

In [ ]:
mols = next(data_iter)
# Extract the data
P = mols['path']
X = mols['coordinates']
E = mols['energies'] #energies are in hartree
S = mols['species']
sm = mols['smiles']

# Print the data
print("Path:   ", P)
print("  Smiles:      ","".join(sm))
print("  Symbols:     ", S)
print("  Coordinates: ", X.shape)
print("  Energies:    ", E.shape, "\n")


How many molecules and conformations are there?

In [ ]:
data_iter = data.__iter__()
count = 0
count_conf = 0
for mol in data_iter:
    count+=1
    count_conf += len(mol['energies'])
print(count)
print(count_conf)

In [ ]:
data.cleanup()

## AEV and implementation
https://github.com/isayev/ASE_ANI.git

### Understanding symmetric functions
Requirements for the representation of atomic environments:
1. Translational invariance
2. Rotational invariance
3. Atomic permutation invariance
![](molecule.jpg)

#### Distance conversion function
$$
f_{\mathrm{C}}\left(R_{i j}\right)=\left\{\begin{array}{cl}0.5 \times \cos \left(\frac{\pi R_{i j}}{R_{\mathrm{C}}}\right)+0.5 & \text { for } R_{i j} \leq R_{\mathrm{C}} \\ 0.0 & \text { for } R_{i j}>R_{\mathrm{C}}\end{array}\right.
$$
Free parameter: $R_C$

Literature values: 
* Radial cutoff   $R_C^{radial}=5.2Å$
* Angular cutoff $R_C^{angular}=3.5Å$

#### Radial environment components
$$
G_{m}^{\mathrm{R}}=\sum_{j \neq i}^{\text {all atoms }} \mathrm{e}^{-\eta\left(R_{i j}-R_{s}\right)^{2}} f_{\mathrm{C}}\left(R_{i j}\right)
$$
Free parameters: $\{\eta, R_s\}$

Literature values:
* $\eta^{radial}=16$
* $R_s^{radial}\in\{0.900000, 1.168750, 1.437500, 1.706250, 1.975000, 2.243750, 2.51250,
 2.781250, 3.050000, 3.318750, 3.587500, 3.856250, 4.125000, 4.39375,
4.662500, 4.931250\}Å$

#### Angular environment components
$$
G_{m}^{A_{\text {mod }}}=2^{1-\zeta} \sum_{j, k \neq i}^{\text {all atoms }}\left(1+\cos \left(\theta_{i j k}-\theta_{s}\right)\right)^{\zeta} \times \exp \left[-\eta\left(\frac{R_{i j}+R_{i k}}{2}-R_{s}\right)^{2}\right] f_{\mathrm{C}}\left(R_{i j}\right) f_{\mathrm{C}}\left(R_{i k}\right)
$$<br>
Free parameters: $\{\zeta, \theta_s, \eta, R_s\}$

Literature values:
* $\zeta=32$
* $\theta_s \in \{0.19634954, 0.58904862,0. 9817477, 1.3744468, 1.7671459,
2.1598449, 2.552544, 2.945243\}$
* $\eta^{angular}=8$
* $R_s^{angular}\in\{0.900000, 1.550000, 2.200000, 2.850000\}Å$

In [ ]:
import numpy as np


def calc_f_C(Rij, RC):
    f_C_value = 0.5 * np.cos(np.pi * Rij / RC) + 0.5
    # Make f_C(0)=0 to make sure the sum in distance conversion function 
    # and radial conversion function can run with j=i
    indicator = ((Rij <= RC) & (Rij != 0)).astype(float) 
    return f_C_value * indicator
    

def radial_component(Rijs, eta, Rs, RC=5.2):
    # Rijs is a 1d array, all other parameters are scalars
    f_C_values = calc_f_C(Rijs, RC)
    individual_components = np.exp(-eta * (Rijs - Rs) ** 2) * f_C_values
    return np.sum(individual_components)

def angular_component(Rij_vectors, Rik_vectors, zeta, theta_s, eta, Rs, RC=3.5):
    # Rij_vectors and Rik_vectors are 2d arrays with shape (n_atoms, 3), all other parameters are scalars
    # calculate theta_ijk values from vector operations
    dot_products = Rij_vectors.dot(Rik_vectors.T)
    Rij_norms = np.linalg.norm(Rij_vectors, axis=-1)
    Rik_norms = np.linalg.norm(Rik_vectors, axis=-1)
    norms = Rij_norms.reshape((-1, 1)).dot(Rik_norms.reshape((1, -1)))
    cos_values = np.clip(dot_products / (norms + 1e-8), -1, 1)
    theta_ijks = np.arccos(cos_values)
    theta_ijk_filter = (theta_ijks != 0).astype(float)
    mean_dists = (Rij_norms.reshape((-1, 1)) + Rik_norms.reshape((1, -1))) / 2
    f_C_values_Rij = calc_f_C(Rij_norms, RC)
    f_C_values_Rik = calc_f_C(Rik_norms, RC)
    f_C_values = f_C_values_Rij.reshape((-1, 1)).dot(f_C_values_Rik.reshape((1, -1)))
    individual_components = (1 + np.cos(theta_ijks - theta_s)) ** zeta * \
        np.exp(-eta * (mean_dists - Rs) ** 2) * f_C_values * theta_ijk_filter
    return 2 ** (1 - zeta) * np.sum(individual_components)

def calc_aev(atom_types, coords, i_index):
    # atom_types are np.array of ints
    relative_coordinates = coords - coords[i_index]
    nearby_atom_indicator = np.linalg.norm(relative_coordinates, axis=-1) < 5.3
    relative_coordinates = relative_coordinates[nearby_atom_indicator]
    atom_types = atom_types[nearby_atom_indicator]
    radial_aev = np.array([radial_component(np.linalg.norm(relative_coordinates[atom_types == atom], 
                                                           axis=-1), eta, Rs) \
                           for atom in [0, 1, 2, 3] for eta in [16] \
                           for Rs in [0.900000,1.168750,1.437500,1.706250,1.975000,2.243750,2.51250,2.781250,3.050000,\
                                   3.318750,3.587500,3.856250,4.125000,4.39375,4.662500,4.931250]])
    angular_aev = np.array([angular_component(relative_coordinates[atom_types == atom_j], 
                                              relative_coordinates[atom_types == atom_k],\
                                             zeta, theta_s, eta, Rs) \
                            for atom_j in [0, 1, 2, 3] for atom_k in range(atom_j, 4) for zeta in [32] \
                            for theta_s in [0.19634954,0.58904862,0.9817477,1.3744468,1.7671459,2.1598449,2.552544,2.945243]\
                            for eta in [8] for Rs in [0.900000,1.550000,2.200000,2.850000]])
    print(len(radial_aev), len(angular_aev))
    return np.concatenate([radial_aev, angular_aev])

        

### Distance Conversion function

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
r_ij = np.linspace(0,10,500)
rc = 5.2
plt.plot(r_ij,[calc_f_C(r,rc)for r in r_ij])

### Visualizing AEVs

In [ ]:
mapping={"H":0, "C":1, "N":2, "O":3}

In [ ]:
elements= np.array([mapping[atom] for atom in S])
elements

Let's plot the AEV for the same atom in different conformation

In [ ]:
plt.plot(calc_aev(elements, X[0], 2), label="conf1", alpha=0.6)
plt.plot(calc_aev(elements, X[10], 2), label="conf2", alpha=0.6, ls="--")
plt.legend()

AEV implemented in pytorch  
https://github.com/aiqm/torchani.git

In [1]:
import torchani
import torch

Rcr = 5.2
Rca = 3.5
EtaR = torch.tensor([16], dtype=torch.float)
ShfR = torch.tensor([0.900000,1.168750,1.437500,1.706250,1.975000,2.243750,2.51250,2.781250,3.050000,\
                            3.318750,3.587500,3.856250,4.125000,4.39375,4.662500,4.931250], dtype=torch.float)
EtaA= torch.tensor([8], dtype=torch.float)
Zeta = torch.tensor([32], dtype=torch.float)
ShfA = torch.tensor([0.900000,1.550000,2.200000,2.850000], dtype=torch.float)
ShfZ = torch.tensor([0.19634954,0.58904862,0.9817477,1.3744468,1.7671459,2.1598449,2.552544,2.945243], 
                    dtype=torch.float)
num_species = 4
aev_computer = torchani.AEVComputer(Rcr, Rca, EtaR, 
                                    ShfR, EtaA, Zeta, ShfA, ShfZ, num_species)

In [2]:
aevs = aev_computer.forward((torch.tensor(elements.reshape(1, -1), dtype=torch.long), 
                             torch.tensor(X[10].reshape(1, -1, 3), dtype=torch.float)))

NameError: name 'elements' is not defined

In [ ]:
plt.plot(aevs[1].numpy()[0, 2])

## Network as a sum of sub-networks using ModuleDict

In [ ]:
class ANI(nn.Module):
    def __init__(self):
        super().__init__()
        self.sub_nets = nn.ModuleDict({"C": ANI_sub([architecture]), "H": ANI_sub([architecture],...)})

    def forward(self, aevs, atom_types):
        atomic_energies = ...
        
        total_energies = torch.sum(atomic_energies,dim=...)
        return total_eneirgies

class ANI_sub(nn.Module):
    def __init__(self, architecture):
        super().__init__()
        ...

    def forward(self, aev):
        atomic_energy = ...
        return atomic_energy


## General suggestions for final project
* Read the paper
    * Atomic "self energies" are subtracted from the molecular energies; thus we are effectively learning the energy difference to a reference state.
* Test the workflow on small amount of data first (up to 2 heavy atoms)
* Tuning hyperparameters and finalizing architecture with a little more data.
* Final production run on full data ( to be defined based on resource to be provided)
* Save intermediate result!!! (https://pytorch.org/tutorials/beginner/saving_loading_models.html)
